In [2]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from tensorflow.keras.models import load_model
import tkinter 
from tkinter import messagebox
import smtplib
import pyttsx3

In [3]:
# Initialize Tkinter
root = tkinter.Tk()
root.withdraw()

# Load trained deep learning model
model = load_model('employee_face_mask_detection_alert_system.h5')

# Classifier to detect face
face_det_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Dictionaries containing details of Wearing Mask and Color of rectangle around face
text_dict = {0:'Mask ON', 1:'No Mask'}
rect_color_dict = {0:(0,255,0), 1:(0,0,255)}

SUBJECT = 'COVID protocol violation'
#TEXT = 'One Visitor violated Face Mask Policy.'


# from PIL import ImageGrab
path = 'ImagesAttendance'
images = []
classNames = []
myList = os.listdir(path)
# print(myList)

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
# print(classNames)

# text-to-speech initialisation
engine = pyttsx3.init()

In [6]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
 
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')
            engine.say('Alert ,'+name+' please wear your mask.')
            engine.runAndWait()
            
encodeListKnown = findEncodings(images)
print('Encoding Complete')

# Capture Video
vid_source = cv2.VideoCapture(0)


# Detect Camera feed
while(True):
    ret, img = vid_source.read()
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_det_classifier.detectMultiScale(grayscale_img, 1.3, 5)
    
    for (x,y,w,h) in faces:
        face_img = grayscale_img[y:y+w, x:x+w]
        resized_img = cv2.resize(face_img, (112,112))
        normalized_img = resized_img/255.0
        reshaped_img = np.reshape(normalized_img, (1,112,112,1))
        result = model.predict(reshaped_img)
        
        label = np.argmax(result, axis=1)[0]
        
        # If label=1 then it means wearing No Mask and 0 means wearing Mask
        
        if label==1:
            imgS = cv2.resize(img,(0,0),None,0.25,0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

            facesCurFrame = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)

            for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
                matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
                faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
                #print(faceDis)
                matchIndex = np.argmin(faceDis)

                if faceDis[matchIndex]< 0.50:
                    name = classNames[matchIndex].upper()
                else: name = 'Unknown'
                #print(name)
                y1,x2,y2,x1 = faceLoc
                y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,0,255),2)
                cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,0,255),cv2.FILLED)
                cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                markAttendance(name)
 
            
                '''
                messagebox.showwarning('Warning','Acess Denied.')

                # Send email to the administrator
                now = datetime.now()
                dt = now.strftime('%H:%M:%S')
                message = 'Subject: {}\n\n{} violated Face Mask Policy at {} today.'.format(SUBJECT, name, dt)
                mail = smtplib.SMTP('smtp.gmail.com', 587)
                mail.ehlo()
                mail.starttls()
                mail.login('email-id','password')
                mail.sendmail('email-id','email-id',message)
                mail.close
                '''
        else:
            cv2.rectangle(img, (x,y), (x+w,y+h), rect_color_dict[label],2)
            cv2.rectangle(img, (x,y-40),(x+w,y), rect_color_dict[label],-1)
            cv2.putText(img, text_dict[label], (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8,(0,0,0),2)
            pass
            break
        
    cv2.imshow('LIVE Video Feed',img)
    key = cv2.waitKey(1)
    
    if key==27:
        break

cv2.destroyAllWindows()
vid_source.release()

Encoding Complete
